In [ ]:
%load_ext dotenv
%dotenv
import os

os.environ["DEBUG"] = "1"  # Set to "1" if you want to use debug mode.
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
from typing import Dict, List, Tuple
from prompttools.experiment import HuggingFaceHubExperiment
print

In [ ]:
from prompttools.experiment import SequentialChainExperiment
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from prompttools.utils import semantic_similarity


llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template("Translate the following review to english:" "\n\n{Review}")

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}"
)

third_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

experiment = SequentialChainExperiment(
    llm=[ChatOpenAI],
    prompt=["أنا أحب أكل الشاورما", "me encanta comer shawarma"],
    prompt_template=[[first_prompt, second_prompt, third_prompt, fourth_prompt]],
    **{
        "temperature": [0.1, 0.9],
        "input_variables": [["Review"]],
        "output_variables": [["English_Review", "summary", "followup_message"]],
        "output_key": [["English_Review", "summary", "language", "followup_message"]],
    },
)


experiment.run()

experiment.evaluate("similar_to_expected", semantic_similarity, expected=["I love to eat shawarma"] * 4)

experiment.visualize()

In [ ]:
models = ["mistralai/Mistral-7B-v0.1"]
prompts = [
    "Two turntables and a?",
    "Three turntables and a?",
]
task = ["text-generation"]
temperatures = [0.0, 1.0]

experiment = HuggingFaceHubExperiment(models, prompts, task, temperature=temperatures)

In [ ]:
experiment.run

In [ ]:
from prompttools.utils import semantic_similarity


In [ ]:
experiment.evaluate("similar_to_expected", semantic_similarity, expected=["microphone"] * 4)


In [ ]:
experiment.visualize()


In [ ]:
# Copyright (c) Hegel AI, Inc.
# All rights reserved.
#
# This source code's license can be found in the
# LICENSE file in the root directory of this source tree.

import os
import jinja2
import prompttools.prompttest as prompttest
from prompttools.utils import similarity
from prompttools.prompttest.threshold_type import ThresholdType
from prompttools.mock.mock import mock_hf_completion_fn
from huggingface_hub.inference_api import InferenceApi

if not (("HUGGINGFACEHUB_API_TOKEN" in os.environ) or ("DEBUG" in os.environ)):
    print("Error: This example requires you to set either your HUGGINGFACEHUB_API_TOKEN or DEBUG=1")
    exit(1)


client = InferenceApi(
    repo_id="google/flan-t5-xxl",
    token=os.environ.get("HUGGINGFACEHUB_API_TOKEN"),
    task="text2text-generation",
)


def create_prompt():
    prompt_template = "Answer the following question: {{ input }}"
    user_input = {"input": "Who was the first president of the USA?"}
    environment = jinja2.Environment()
    template = environment.from_string(prompt_template)
    return template.render(**user_input)


@prompttest.prompttest(
    metric_name="similar_to_expected",
    eval_fn=similarity.evaluate,
    prompts=[create_prompt()],
    expected=["George Washington"],
    threshold=1.0,
    threshold_type=ThresholdType.MAXIMUM,
)
def completion_fn(prompt: str):
    response = None
    if os.getenv("DEBUG", default=False):
        response = mock_hf_completion_fn(**{"inputs": prompt})
    else:
        response = client(inputs=prompt)
    return response[0]["generated_text"]


if __name__ == "__main__":
    prompttest.main()